# Simulating ethanol protein production in Saccharomyces cerevisiae with COBRApy

This notebook provides an introduction to constraint-based metabolic modelling with COBRApy, followed by several practical exercisesm where you will be expected to fill out the missing code.

We are going to work with a genome-scale metabolic model of yeast: [iMM904](http://bigg.ucsd.edu/models/iMM904).


## COBRApy
COBRApy is a python package for constraint-based metabolic modelling. It contains functions for running various analyses, such as FBA, pFBA, FVA, flux sampling, building and editing models...
Here you can find the [COBRApy documentation](https://cobrapy.readthedocs.io/en/latest/).  
In case you prefer MATLAB, there is [COBRA Toolbox](https://opencobra.github.io/cobratoolbox/stable/). It has additional functionalities compared to COBRApy (e.g. omics integration).

COBRApy can read various model file types. We will use **SBML** (Systems Biology Markup Language) - one of the standard formats how to store a model. The advantage of this format is that it is both human an machine readable. This is what a reaction looks like:

<img src="figs/sbml_rxn.png" alt="SBML reaction" style="width: 1000px;"/>


Metabolic models are often deposited in databases such as [BiGG Models](http://bigg.ucsd.edu/) or [BioModels](https://www.ebi.ac.uk/biomodels/).



In [1]:
# import necessary packages
import cobra  # cobrapy package
import escher  # package for visualization
import numpy as np  # many useful math & data analysis functions
import matplotlib.pyplot as plt  # plotting

In [2]:
# read the model
model = cobra.io.read_sbml_model("models/iMM904.xml")

In [3]:
# basic overview of the model
model

Name,iMM904
Memory address,0x07fe6f5e3ab90
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [4]:
# check what compartments the model has
model.compartments

{'c': 'cytosol',
 'e': 'extracellular space',
 'm': 'mitochondria',
 'x': 'peroxisome/glyoxysome',
 'r': 'endoplasmic reticulum',
 'v': 'vacuole',
 'g': 'golgi apparatus',
 'n': 'nucleus'}

In [5]:
# access metabolites
model.metabolites[:10]  # show only the first ten metabolites

[<Metabolite 2dr5p_c at 0x7fe7646aa150>,
 <Metabolite 2hb_c at 0x7fe6f5de73d0>,
 <Metabolite 2hb_e at 0x7fe6f5ea7e10>,
 <Metabolite 2hhxdal_c at 0x7fe6f5e07e10>,
 <Metabolite 2hp6mbq_m at 0x7fe72c696510>,
 <Metabolite 2hp6mp_m at 0x7fe72c6968d0>,
 <Metabolite 2hpmhmbq_m at 0x7fe72c696a90>,
 <Metabolite 2hpmmbq_m at 0x7fe72c696f50>,
 <Metabolite 2ippm_c at 0x7fe72c697090>,
 <Metabolite 2kmb_c at 0x7fe72c696950>]

In [6]:
# access one metabolite with its ID and save it as a variable 
glucose = model.metabolites.get_by_id("glc__D_c") 
# model.metabolites.glc__D_c - shorter syntax, but does not work if metabolite ID starts with a number / contains brackets
glucose

Metabolite identifier,glc__D_c
Name,D-Glucose
Memory address,0x07fe72c4db210
Formula,C6H12O6
Compartment,c
In 11 reaction(s),"MALT, GLCGSD, GLCt1, SBTR, 13BGH, GLUK, GALS3, GLCtv, HEX1, TREH, DGGH"


In [7]:
# metabolite ID - unique identifier
glucose.id

'glc__D_c'

In [8]:
# metabolite name - not unique (one metabolite can be present in several compartments)
glucose.name

'D-Glucose'

In [9]:
# formula
glucose.formula

'C6H12O6'

In [10]:
# database identifiers
glucose.annotation

{'sbo': 'SBO:0000247',
 'bigg.metabolite': 'glc__D',
 'biocyc': 'META:Glucopyranose',
 'chebi': ['CHEBI:12965', 'CHEBI:20999', 'CHEBI:4167', 'CHEBI:17634'],
 'hmdb': ['HMDB00122', 'HMDB06564'],
 'inchi_key': 'WQZGKKKJIJFFOK-GASJEMHNSA-N',
 'kegg.compound': 'C00031',
 'kegg.drug': 'D00009',
 'metanetx.chemical': 'MNXM41',
 'sabiork': ['1406', '1407'],
 'seed.compound': ['cpd26821', 'cpd00027']}

In [11]:
# list of participating reactions
glucose.reactions

frozenset({<Reaction 13BGH at 0x7fe72c0d2f10>,
           <Reaction DGGH at 0x7fe72c10e7d0>,
           <Reaction GALS3 at 0x7fe72ba8c310>,
           <Reaction GLCGSD at 0x7fe72b9b8c90>,
           <Reaction GLCt1 at 0x7fe72b95fed0>,
           <Reaction GLCtv at 0x7fe72b95ffd0>,
           <Reaction GLUK at 0x7fe72babcf10>,
           <Reaction HEX1 at 0x7fe72b90cdd0>,
           <Reaction MALT at 0x7fe72b7cb210>,
           <Reaction SBTR at 0x7fe72b3fecd0>,
           <Reaction TREH at 0x7fe72b34c990>})

In [12]:
# access model reactions 
model.reactions[:10]  # print only first ten

[<Reaction CITtcp at 0x7fe72c0d2f90>,
 <Reaction 13BGH at 0x7fe72c0d2f10>,
 <Reaction 13BGHe at 0x7fe72c0d4510>,
 <Reaction 13GS at 0x7fe72c0d4e90>,
 <Reaction 16GS at 0x7fe72c0e0450>,
 <Reaction 23CAPPD at 0x7fe72c0e0e50>,
 <Reaction 2DDA7Ptm at 0x7fe72c0e0090>,
 <Reaction 2DHPtm at 0x7fe72c0d4a50>,
 <Reaction 2DOXG6PP at 0x7fe72c0e0150>,
 <Reaction 2HBO at 0x7fe72c0e8590>]

In [13]:
# access one reaction
hexokinase = model.reactions.HEX1  # (or model.reactions.get_by_id("HEX1"))
hexokinase

Reaction identifier,HEX1
Name,Hexokinase (D-glucose:ATP)
Memory address,0x07fe72b90cdd0
Stoichiometry,atp_c + glc__D_c --> adp_c + g6p_c + h_c ATP C10H12N5O13P3 + D-Glucose --> ADP C10H12N5O10P2 + D-Glucose 6-phosphate + H+
GPR,YFR053C or YGL253W or YCL040W
Lower bound,0.0
Upper bound,999999.0


In [14]:
# participating metabolites - negative stoichiometric coefficient - reactants; positive - products
hexokinase.metabolites

{<Metabolite atp_c at 0x7fe72c5b41d0>: -1.0,
 <Metabolite glc__D_c at 0x7fe72c4db210>: -1.0,
 <Metabolite adp_c at 0x7fe72c621050>: 1.0,
 <Metabolite g6p_c at 0x7fe72c52c210>: 1.0,
 <Metabolite h_c at 0x7fe72c49b210>: 1.0}

In [15]:
# upper bound (UB) of the reaction flux - maximum possible flux
# typically set to an arbitrary value like 999999 or 1000
# this represents an unlimited flux (we cannot set it to infinity, so we use arbitrarily high values)
hexokinase.upper_bound

999999.0

In [16]:
# lower bound (LB) of the reaction flux
# irreversible reactions have LB=0, reversible have LB=-999999 
hexokinase.lower_bound

0.0

In [17]:
# check if the reaction is reversible (you can also see it from the lower bound)
hexokinase.reversibility

False

## Important reactions
### Exchange reactions

Exchange reactions are arbitrary reactions that let metabolites enter and leave the modelling system. Their lower and upper bounds can be changed according to experimentally measured uptake and secretion rates.

The exchange reactions have the form "M_e -> " or "M_e <-> ". There is a reactant but no product. 

The model also contains the individual transporter reactions, e.g. "M_c -> M_e" - this represents a transporter that transports metabolite M from cytoplasm to extracellular space. 

So why do we need the exchange reactions?

* Steady state assumption. Metabolites cannot accumulate (Sv=0), so we need reactions that allow metabolites to enter and leave the system.

* There are often several transporters for the same metabolite and one transporter can often transport several types of molecules. But when we measure exchange rates in the lab, we don't know which transporter is doing the job, we just see the overall rate. Exchange reactions conveniently allow us to set the overall rate and we don't need to specify the rates for the individual transporters.

In [18]:
# list of exchange reactions. IDs typically start with "EX_"
model.exchanges[:10]

[<Reaction EX_epistest_SC_e at 0x7fe72c0e7150>,
 <Reaction EX_epist_e at 0x7fe72c0f8c50>,
 <Reaction EX_ergst_e at 0x7fe72bc69d10>,
 <Reaction EX_ergstest_SC_e at 0x7fe72bc69ed0>,
 <Reaction EX_13BDglcn_e at 0x7fe72bc6d2d0>,
 <Reaction EX_etha_e at 0x7fe72bc6d490>,
 <Reaction EX_2hb_e at 0x7fe72bc6d110>,
 <Reaction EX_etoh_e at 0x7fe72bc73390>,
 <Reaction EX_fe2_e at 0x7fe72bc6dc50>,
 <Reaction EX_fecost_e at 0x7fe72bc73910>]

**Glucose uptake rate**  
Notice how the reaction is defined. If it is going forward, glucose leaves the system - it is secreted. Reverse direction indicates uptake.
In this model, glucose has a default LB = -10. This means the maximum uptake rate is 10 mmol/(gDW*h)

In [19]:
model.reactions.EX_glc__D_e 

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x07fe72bc16090
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,999999.0


### Growth medium
Growth medium typically contains one or more carbon sources, nitrogen source, water and various ions. 

The values are **lower bounds** of the uptake rates [mmol/gDW h]. Some metabolites have a default value (e.g. glucose and O2)  based on measurements. Others have an unlimited uptake (value of 999999), meaning that the cells can take up as much as they want.
Note that they are shown as abs. values (if you look at the LB and UB of exchange reactions, uptake rates have negative value, secretion rates positive).

In [20]:
# Medium components that cells can take up. 
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

In [21]:
# display reaction names so we know what is what
for element in model.medium:
    print(f"{element}, {model.reactions.get_by_id(element).name, model.medium[element]}")

EX_fe2_e, ('Fe2+ exchange', 999999.0)
EX_glc__D_e, ('D-Glucose exchange', 10.0)
EX_h2o_e, ('H2O exchange', 999999.0)
EX_h_e, ('H+ exchange', 999999.0)
EX_k_e, ('K+ exchange', 999999.0)
EX_na1_e, ('Sodium exchange', 999999.0)
EX_so4_e, ('Sulfate exchange', 999999.0)
EX_nh4_e, ('Ammonia exchange', 999999.0)
EX_o2_e, ('O2 exchange', 2.0)
EX_pi_e, ('Phosphate exchange', 999999.0)


### Biomass reaction

An artificial reaction that represents a drain of metabolites towards growth. It represents the cellular composition - the amount of amino acids in proteins, DNA, RNA, lipids, carbohydrates, ATP...

In [22]:
model.reactions.BIOMASS_SC5_notrace

Reaction identifier,BIOMASS_SC5_notrace
Name,Biomass SC5 notrace
Memory address,0x07fe72b2c2590
Stoichiometry,1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024... 1.1348 1 3 beta D Glucan C6H10O5 + 0.4588 L-Alanine + 0.046 AMP C10H12N5O7P + 0.1607 L-Arginine + 0.1017 L-Asparagine + 0.2975 L-Aspartate + 59.276 ATP C10H12N5O13P3 + 0.0447 CMP C9H12N3O8P +...
GPR,
Lower bound,0.0
Upper bound,999999.0


### ATP maintenance reaction
Another artificial reaction which represents non-growth associated maintenance energy. This is energy that cells need just to survive (even if they do not grow). It represents energy use by processes like maintenance of chemical gradients or macromolecule turnover. It was shown that [it improves flux predictions if it is included as a model parameter.](https://doi.org/10.1371/journal.pcbi.1009022)

In [23]:
model.reactions.ATPM 

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x07fe72be81050
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,
Lower bound,1.0
Upper bound,1.0


## Genes
Typically models have **gene protein reaction (GPR) rules** - associations of reactions with genes.
One reaction is often associated with several genes and the relationship is expressed with **AND** / **OR** rules.
* **AND** rule - e.g. **Gene1 AND Gene2** - both genes are needed to catalyze reaction (e.g. two subunits)
* **OR** rule - **Gene1 OR Gene2** - isoenzymes

Also combinations are possible, e.g. **(Gene1 AND Gene2) OR (Gene1 AND Gene3)**.   

One gene often participates in several reactions - e.g. promiscuous enzymes.

In [24]:
# access model genes
model.genes[:10]

[<Gene YHR104W at 0x7fe72c2d8990>,
 <Gene YDR368W at 0x7fe72c2a8090>,
 <Gene YGR282C at 0x7fe72c2a8390>,
 <Gene YOL086C at 0x7fe72c2a8650>,
 <Gene YLR300W at 0x7fe72c2a8910>,
 <Gene YFR055W at 0x7fe72c2a8c50>,
 <Gene YGL184C at 0x7fe72c2b8210>,
 <Gene YDR261C at 0x7fe72c2b8390>,
 <Gene YDL168W at 0x7fe72c2b8650>,
 <Gene YOR190W at 0x7fe72c2a8f90>]

In [25]:
# examine GPR rule for hexokinase
hexokinase.gene_reaction_rule

'YFR053C or YGL253W or YCL040W'

In [26]:
# for a given gene, find all reactions where it occurs
model.genes.get_by_id("YHR104W").reactions

frozenset({<Reaction ALCD19y at 0x7fe72c0237d0>,
           <Reaction ARABR at 0x7fe72bf31e90>,
           <Reaction LALDO3 at 0x7fe72b7d1f90>,
           <Reaction SBTR at 0x7fe72b3fecd0>,
           <Reaction XYLR at 0x7fe72b302f50>})

## Looking up stuff
If you don't know the name of your metabolite/reaction/gene, you can search for certain words in the model.  
Note: the search is case sensitive.

In [27]:
# you can search for the metabolite name
model.metabolites.query("Glucose", "name")

[<Metabolite 6dg_c at 0x7fe72c673050>,
 <Metabolite g1p_c at 0x7fe72c51e5d0>,
 <Metabolite g6p_c at 0x7fe72c52c210>,
 <Metabolite g6p_r at 0x7fe72c52e790>,
 <Metabolite glc__D_c at 0x7fe72c4db210>,
 <Metabolite glc__D_e at 0x7fe72c4db950>,
 <Metabolite glc__D_v at 0x7fe72c4deb50>]

In [28]:
# search for all metabolites with a given formula
model.metabolites.query("C6H12O6", "formula")

[<Metabolite 14glun_c at 0x7fe72c6b4990>,
 <Metabolite fru_c at 0x7fe72c515990>,
 <Metabolite fru_e at 0x7fe72c515f90>,
 <Metabolite gal_c at 0x7fe72c533450>,
 <Metabolite gal_e at 0x7fe72c535150>,
 <Metabolite glc__D_c at 0x7fe72c4db210>,
 <Metabolite glc__D_e at 0x7fe72c4db950>,
 <Metabolite glc__D_v at 0x7fe72c4deb50>,
 <Metabolite inost_c at 0x7fe72c4acc10>,
 <Metabolite inost_e at 0x7fe72c4af610>,
 <Metabolite man_c at 0x7fe72c4c1050>,
 <Metabolite man_e at 0x7fe72c4c41d0>,
 <Metabolite srb__L_c at 0x7fe72c39d950>,
 <Metabolite srb__L_e at 0x7fe72c3a0210>]

In [29]:
# reaction name
model.reactions.query("Hexokinase", "name")

[<Reaction HEX1 at 0x7fe72b90cdd0>,
 <Reaction HEX4 at 0x7fe72b90c610>,
 <Reaction HEX7 at 0x7fe72b90cf90>]

## Changing constraints
Often we measure uptake and secretion rates of metabolites when performing cell cultivations. E.g. we might measure glucose uptake and ethanol secretion rate. We integrate these into the model by changing the **lower and upper bounds of the exchange reactions**.

Typically, the units of the exchange reactions are mmol/(gDW h).
Negative values - uptake, positive - secretion.

In [30]:
# changing constraints
glc_uptake = model.reactions.get_by_id("EX_glc__D_e")
glc_uptake

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x07fe72bc16090
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,999999.0


In [31]:
# disable glucose uptake
glc_uptake.lower_bound = 0  # or directly model.reactions.EX_glc__D_e.lower_bound = 0
glc_uptake

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x07fe72bc16090
Stoichiometry,glc__D_e --> D-Glucose -->
GPR,
Lower bound,0
Upper bound,999999.0


In [32]:
# set it back to original value
glc_uptake.lower_bound = -10

Often you will test various conditions (e.g. various carbon sources, various levels of substrates etc...).
You always need to make sure to reset the values back to the original value.
A very helpful function for this is **with statement**. *What happens in with statement stays in with statement.*

In [33]:
with model:
    # everything that happens here in the indented block is reset afterwards
    glc_uptake.lower_bound = -5
    print(f"Uptake rate inside 'with' statement: {glc_uptake.lower_bound}")
    
print(f"Uptake rate outside 'with' statement: {glc_uptake.lower_bound}")

Uptake rate inside 'with' statement: -5
Uptake rate outside 'with' statement: -10


**Note**  
Normally in jupyter notebooks, if something is on the last line, it will be automatically printed (e.g. if you just type ```glc_uptake.lower_bound```). If you want to print something inside of a "with statement" or a "for loop", you need to put it inside of a ```print()``` function. 

## Performing simulations

Before running FBA we have to choose an **objective function**, that we want to maximize or minimize. Typically this is maximiation of biomass production. Another option would be maximization of secretion of a certain product (e.g. ethanol).

In [34]:
# Set the objective. Biomass reaction - a typical objective.   
model.objective = "BIOMASS_SC5_notrace"

In [35]:
# Perform FBA
fba = model.optimize("maximize")
fba
# Reduced costs - impact of an increase of the flux through the respective reaction on the final objective value

,fluxes,reduced_costs
CITtcp,0.000000,-3.469447e-18
13BGH,0.000000,-4.751566e-02
13BGHe,0.000000,0.000000e+00
13GS,0.326670,0.000000e+00
16GS,0.000000,-9.503132e-02
...,...,...
PYDXO,0.000000,0.000000e+00
PYK,17.721990,0.000000e+00
PYNP2r,0.000000,-0.000000e+00
PYR5CDm,0.000000,-2.969729e-02


In [36]:
# access the objective value (growth rate in  1/h)
fba.objective_value

0.28786570370401793

In [37]:
# access fluxes (in mmol/(gDW*h))
fba.fluxes

CITtcp      0.000000
13BGH       0.000000
13BGHe      0.000000
13GS        0.326670
16GS        0.000000
             ...    
PYDXO       0.000000
PYK        17.721990
PYNP2r      0.000000
PYR5CDm     0.000000
PYRDC      15.946051
Name: fluxes, Length: 1577, dtype: float64

In [38]:
# get a nice summary with you objective and uptake/secretion rates
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.611,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05691,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.64,0,0.00%


In [39]:
# this command also runs FBA, but the only output is the objective value
# advantage - faster (can be useful if you need to run thousands of simulations)
model.slim_optimize()

0.28786570370401565

In [40]:
# If you need more info about a function, put a question mark after the command
model.slim_optimize?

Signature: model.slim_optimize(error_value=nan, message=None)
Docstring:
Optimize model without creating a solution object.

Creating a full solution object implies fetching shadow prices and
flux values for all reactions and metabolites from the solver
object. This necessarily takes some time and in cases where only one
or two values are of interest, it is recommended to instead use this
function which does not create a solution object returning only the
value of the objective. Note however that the `optimize()` function
uses efficient means to fetch values so if you need fluxes/shadow
prices for more than say 4 reactions/metabolites, then the total
speed increase of `slim_optimize` versus `optimize` is  expected to
be small or even negative depending on how you fetch the values
after optimization.

Parameters
----------
error_value : float, None
   The value to return if optimization failed due to e.g.
   infeasibility. If None, raise `OptimizationError` if the
   optimization fails.

## Knock-outs
Why simulate knock-outs?
* Determine whether a reaction is essential for growth/other functions
* Improve properties of a cell (e.g. remove reactions that secrete waste products)
* Test different genotypes (e.g. drug sensitivity of cancer cells with different mutations) 

We can either knock out reactions or genes.

In [41]:
# Reaction knock-out
with model:
    model.reactions.CYOR_u6m .knock_out()
    print(f"Growth rate after KO: {model.slim_optimize()}") 
    
# outside "with" statement, the model is back to normal
print(f"Wild type growth rate {model.slim_optimize()}")

Growth rate after KO: 0.20932926167108812
Wild type growth rate 0.28786570370401576


In [42]:
# Gene knock-out
with model:
    model.genes.get_by_id("YBL045C").knock_out()
    print(f"Growth rate after KO: {model.slim_optimize()}") 
    
# outside "with" statement, the model is back to normal
print(f"Wild type growth rate {model.slim_optimize()}")

Growth rate after KO: 0.20932926167108826
Wild type growth rate 0.28786570370401693


# Exercises
First we will define our growth medium. The default medium is minimal - it contains one carbon source (glucose), one nitrogen source (ammonium), H2O and ions.

For anaerobic growth we also need lipids in the medium.

In [43]:
# we do not have data on uptake rates - we can set lower bound to -1 (same as glucose)
# it is unlikely that the uptake of lipids would be higher than glucose
model.reactions.EX_ergstest_SC_e.lower_bound = -10
model.reactions.EX_zymstest_SC_e.lower_bound = -10
model.reactions.EX_ptd1ino_SC_e.lower_bound = -10

# check what is in the medium
model.medium

{'EX_ergstest_SC_e': 10,
 'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0,
 'EX_ptd1ino_SC_e': 10,
 'EX_zymstest_SC_e': 10}

## Exercise 1 - FBA in aerobic conditions
What is the maximum growth rate under these conditions? Try changing the value of the glucose uptake rate. Save the predicted fluxes (we will need them in exercise 6).

In [44]:
# Your code here

The value of the uptake rate greatly influcences predictions - this shows importance of uptake rate measurements on the predictions. If the medium contains more than one carbon source, ideally all of them should be measured (or at least the most abundand ones).

## Exercise 2 - FBA in anaerobic conditions
What is the maximum growth rate in anaerobic conditions (under the same glucose uptake rate as before)? Save the predicted fluxes (we will need them in exercise 6).

In [45]:
# Your code here

## Exercise 3 - ethanol production envelope in aerobic conditions
You are interested in ethanol production capabilities of your strain. To analyse ethanol production at different growth rates, make a production envelope: 
* make a list of growth rates from zero to maximum possible
* go over the list of growth rates step by step and use them as a constraint (lower bound)
* set ethanol as the objective and minimize & maximize its secretion and save these values
* plot your results (x-axis - growth rate, y-axis - min. and max. ethanol secretion)

In [46]:
# make two empty lists that will be used to store the maximum and minimum values of ethanol production


# make a list of tested growth rates from 0 to maximum from exercise 1 (Hint: use the function np.linspace())


# loop over the list of growth rates

    # use with statement

        # set glucose uptake 
        
        # set biomass to the tested growth rate ('mu')
        
        # set objective to ethanol exchange
        
        # maximize and minimize ethanol production and add the values to the empty lists



In [47]:
# plot solution space - these are (theoretically) all possible growth rates and ethanol secretion rates that cell can achieve
# uncomment following lines and replace the variable names with your names
# plt.plot(mus, max_etoh, color = "blue")
# plt.plot(mus, min_etoh, color = "blue")
# plt.xlabel("Growth rate [1/h]")
# plt.ylabel("Ethanol productivity [mmol/gDW h]")
# plt.fill_between(mus, min_etoh, max_etoh, color = "blue", alpha = 0.3)

The plot above shows all possible growth rate and ethanol secretion values. Up to growth rate of around 0.23, ethanol secretion can be anywhere between 0 and the maximum value. Above this growth rate, ethanol always has to be produced - this phenomenon is called **growth coupling**. In this case, it is a **weak coupling**, because there are growth rates where ethanol production can be theoretically zero.

## Exercise 4 - ethanol production envelope in anaerobic conditions
Let's try the same in anaerobic conditions.

In [48]:
# your code here - same as before, just different conditions


We observe stronger growth coupling - above the growth rate of around 0.14, cells have to produce ethanol.

## Exercise 5 - ethanol production envelope after KOs in anaerobic conditions
A common way how to achieve growth coupling is by making certain knock-out(s). These ensure that the cell has to produce the product of interest (basically because all other alternative pathways are gone). There are algorithms which can predict these, such as **OptKnock, RobustKnock, OptCouple**... However, they are computationally demanding (take a long time to run, especially if you are looking for double, triple, quadruple... KOs, because they have to test all possible combinations of KOs).

Here we are going to try effects of KOs that were previously idendified in [von Kamp, A., & Klamt, S. (2017)](https://dx.doi.org/10.1038%2Fncomms15956): Glycerol 3 phosphate dehydrogenase (NAD), Malate dehydrogenase and NADH dehydrogenase.

In [49]:
# reactions to be knocked out
KOs = ["G3PD1ir", "MDH", "NADH2_u6cm"]

In [50]:
# your code here - similar as before

In [51]:
# plot solution space

We see that after making the konck-outs we observe much stronger growth coupling. When the cells grow above 0.01, they have to produce ethanol. 

Can you think of an explanation why we observe growth coupling after making these knock-outs?

## Exercise 6 - Visualization
In the last part of the notebook we are going to visualize the predicted fluxes and compare flux distributions in aerobic vs anaerobic conditions.

In [52]:
# available maps for different models
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [53]:
# use FBA results from exercise 1 and 2 (replace "fba1.fluxes" with whatever you named your results in the first exercise)
# these commands will create html files which you can open in a browser
# aerobic

# builder = escher.Builder('iMM904.Central carbon metabolism',
#                        reaction_data=fba1.fluxes)
# builder.save_html(f"FBA_aerobic.html")

# anaerobic